In [ ]:
import json
!pip3 install --upgrade typing_extensions --break-system-packages
!pip3 install torch torch-geometric pandas --break-system-packages

In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import pandas as pd
import numpy as np


In [52]:
def prepare_graph(associations_df):
    # Unique nodes
    job_ids = sorted(associations_df['job_id'].unique())
    route_keys = sorted(associations_df['route_key'].unique())

    # Create mappings
    job_id_map = {job_id: idx for idx, job_id in enumerate(job_ids)}
    route_map = {route: idx + len(job_ids) for idx, route in enumerate(route_keys)}

    # Node features
    job_features = []
    for job_id in job_ids:
        job_data = associations_df[associations_df['job_id'] == job_id].iloc[0]
        job_features.append([
            float(job_data['source_cpu']),
            float(job_data['source_ram']),
            float(job_data['job_deadline']),
            float(job_data['transfer_time_hours'])
        ])

    route_features = []
    for route in route_keys:
        route_data = associations_df[associations_df['route_key'] == route].iloc[0]
        route_features.append([
            float(route_data['transfer_time']),
            float(route_data['throughput_gbps']),
            float(route_data['carbon_emissions']),
            float(route_data['source_nic_speed'] == '10Gbps')
        ])

    # Edges
    edges = []
    edge_attrs = []
    for _, row in associations_df.iterrows():
        src = job_id_map[row['job_id']]
        dst = route_map[row['route_key']]
        edges.append((src, dst))
        edge_attrs.append([
            row['transfer_time'],
            row['throughput_gbps'],
            row['carbon_emissions']
        ])

    # Convert to tensors
    x = torch.tensor(job_features + route_features, dtype=torch.float)
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attrs, dtype=torch.float)

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr), job_id_map, route_map

In [53]:
class TemporalGNN(nn.Module):
    def __init__(self, node_in_dim=4, edge_in_dim=3, hidden_dim=64):
        super().__init__()
        self.conv1 = GCNConv(node_in_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)

        # Edge scoring head
        self.edge_head = nn.Sequential(
            nn.Linear(2 * hidden_dim + edge_in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        # Node embeddings
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))

        # Edge scores
        src, dst = edge_index
        edge_features = torch.cat([x[src], x[dst], edge_attr], dim=1)
        scores = self.edge_head(edge_features).squeeze(-1)

        return scores

In [54]:
def train_model(model, data, associations_df, job_id_map, route_map, epochs=100):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # Create meaningful targets
    edge_targets = torch.zeros(data.edge_index.size(1))

    # For each job, find the best route (lowest carbon that meets deadline)
    for job_id, job_idx in job_id_map.items():
        job_data = associations_df[associations_df['job_id'] == job_id]
        min_carbon = job_data['carbon_emissions'].min()
        deadline = job_data['job_deadline'].iloc[0]

        # Get all edges for this job
        edge_mask = (data.edge_index[0] == job_idx)
        edge_indices = edge_mask.nonzero().squeeze()

        # Score each edge
        for edge_idx in edge_indices:
            route_idx = data.edge_index[1, edge_idx].item()
            route_key = [k for k, v in route_map.items() if v == route_idx][0]
            edge_data = job_data[job_data['route_key'] == route_key].iloc[0]

            # Target score based on carbon and deadline feasibility
            carbon_norm = 1 - (edge_data['carbon_emissions'] - min_carbon) / job_data['carbon_emissions'].std()
            feasible = 1.0 if edge_data['transfer_time_hours'] <= deadline else 0.0
            edge_targets[edge_idx] = 0.7 * carbon_norm + 0.3 * feasible

    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        scores = model(data)
        loss = F.mse_loss(scores, edge_targets)
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

In [55]:
from collections import defaultdict


def generate_schedule(model, data, associations_df, job_id_map, route_map):
    model.eval()
    with torch.no_grad():
        scores = model(data)

        # Get best route for each job
        schedule = []
        route_availability = defaultdict(float)

        # Process jobs in deadline order
        jobs_sorted = sorted(job_id_map.keys(),
                             key=lambda x: associations_df[associations_df['job_id'] == x]['job_deadline'].iloc[0])

        for job_id in jobs_sorted:
            job_idx = job_id_map[job_id]
            edge_mask = (data.edge_index[0] == job_idx)
            edge_indices = edge_mask.nonzero().squeeze()

            if len(edge_indices) == 0:
                continue

            # Get top scoring edge
            best_edge = edge_indices[scores[edge_indices].argmax()]
            route_idx = data.edge_index[1, best_edge].item()
            route_key = [k for k, v in route_map.items() if v == route_idx][0]

            # Get job data
            job_data = associations_df[(associations_df['job_id'] == job_id) &
                                       (associations_df['route_key'] == route_key)].iloc[0]

            # Schedule job
            start_time = max(route_availability[route_key], 0)
            end_time = start_time + job_data['transfer_time_hours']

            if end_time <= job_data['job_deadline']:
                schedule.append({
                    'job_id': job_id,
                    'route_key': route_key,
                    'start_time': start_time,
                    'end_time': end_time,
                    'duration': job_data['transfer_time_hours'],
                    'carbon_emissions': job_data['carbon_emissions'],
                    'throughput_gbps': job_data['throughput_gbps']
                })
                route_availability[route_key] = end_time

    return pd.DataFrame(schedule)

In [59]:
associations_df = pd.read_csv("data/associations_df.csv")
data, job_id_map, route_map = prepare_graph(associations_df)
model = TemporalGNN()
train_model(model, data, associations_df, job_id_map, route_map, epochs=1000)

Epoch 0, Loss: 5.2407
Epoch 10, Loss: 5.2407
Epoch 20, Loss: 5.2407
Epoch 30, Loss: 5.2407
Epoch 40, Loss: 5.2407
Epoch 50, Loss: 5.2407
Epoch 60, Loss: 5.2407
Epoch 70, Loss: 5.2407
Epoch 80, Loss: 5.2407
Epoch 90, Loss: 5.2407
Epoch 100, Loss: 5.2407
Epoch 110, Loss: 5.2407
Epoch 120, Loss: 5.2407
Epoch 130, Loss: 5.2407
Epoch 140, Loss: 5.2407
Epoch 150, Loss: 5.2407
Epoch 160, Loss: 5.2407
Epoch 170, Loss: 5.2407
Epoch 180, Loss: 5.2407
Epoch 190, Loss: 5.2407
Epoch 200, Loss: 5.2407
Epoch 210, Loss: 5.2407
Epoch 220, Loss: 5.2407
Epoch 230, Loss: 5.2407
Epoch 240, Loss: 5.2407
Epoch 250, Loss: 5.2407
Epoch 260, Loss: 5.2407
Epoch 270, Loss: 5.2407
Epoch 280, Loss: 5.2407
Epoch 290, Loss: 5.2407
Epoch 300, Loss: 5.2407
Epoch 310, Loss: 5.2407
Epoch 320, Loss: 5.2407
Epoch 330, Loss: 5.2407
Epoch 340, Loss: 5.2407
Epoch 350, Loss: 5.2407
Epoch 360, Loss: 5.2407
Epoch 370, Loss: 5.2407
Epoch 380, Loss: 5.2407
Epoch 390, Loss: 5.2407
Epoch 400, Loss: 5.2407
Epoch 410, Loss: 5.2407
Epo

In [58]:
schedule = generate_schedule(model, data, associations_df, job_id_map, route_map)
schedule.to_csv('schedules/gnn_temporal_schedule.csv')
print("Optimized Schedule:")
print(schedule[['job_id', 'route_key', 'start_time', 'end_time', 'carbon_emissions']])
print(f"\nTotal jobs scheduled: {len(schedule)}")
print(f"Average carbon emissions: {schedule['carbon_emissions'].mean():.2f}")

Optimized Schedule:
     job_id route_key  start_time   end_time  carbon_emissions
0        12  chi_buff    0.000000   0.002888          0.753953
1        20  chi_buff    0.002888   0.005805          0.761444
2        25  chi_buff    0.005805   0.008696          0.754853
3        31  chi_buff    0.008696   0.011585          0.753955
4        36  chi_buff    0.011585   0.014473          0.754055
..      ...       ...         ...        ...               ...
140      70  chi_buff   16.546998  16.644113         26.020229
141     104  chi_buff   16.644113  16.755293         29.791983
142      14  chi_buff   16.755293  23.613734       1859.153729
143      10  chi_buff   23.613734  23.920992         82.369506
144      59  chi_buff   23.920992  24.026950         28.391605

[145 rows x 5 columns]

Total jobs scheduled: 145
Average carbon emissions: 44.80


In [72]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GATv2Conv
from torch_geometric.data import Data
import pandas as pd
from collections import defaultdict


class ScheduleGNN(nn.Module):
    def __init__(self, node_feat_dim=2, edge_feat_dim=3, hidden_dim=128):
        super().__init__()
        # Node encoders
        self.job_encoder = nn.Sequential(
            nn.Linear(node_feat_dim, hidden_dim),
            nn.ReLU()
        )
        self.route_encoder = nn.Sequential(
            nn.Linear(node_feat_dim, hidden_dim),
            nn.ReLU()
        )

        # Graph attention
        self.conv1 = GATv2Conv(hidden_dim, hidden_dim, edge_dim=edge_feat_dim)
        self.conv2 = GATv2Conv(hidden_dim, hidden_dim, edge_dim=edge_feat_dim)

        # Scoring network
        self.scorer = nn.Sequential(
            nn.Linear(2 * hidden_dim + edge_feat_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, data):
        # Encode nodes
        h_jobs = self.job_encoder(data.x[data.job_mask])
        h_routes = self.route_encoder(data.x[data.route_mask])

        # Combine features
        x = torch.zeros(data.num_nodes, h_jobs.size(1), device=data.x.device)
        x[data.job_mask] = h_jobs
        x[data.route_mask] = h_routes

        # Graph processing
        x = F.relu(self.conv1(x, data.edge_index, data.edge_attr))
        x = F.relu(self.conv2(x, data.edge_index, data.edge_attr))

        # Score edges
        src, dst = data.edge_index
        edge_feats = torch.cat([x[src], x[dst], data.edge_attr], dim=1)
        return self.scorer(edge_feats).squeeze()


def prepare_gnn_data(associations_df):
    """Convert dataframe to graph for GNN processing"""
    # Create nodes
    job_nodes = []
    job_map = {}
    for i, job_id in enumerate(associations_df['job_id'].unique()):
        deadline = associations_df[associations_df['job_id'] == job_id]['job_deadline'].iloc[0]
        job_nodes.append([deadline, 0])  # [deadline, current_time]
        job_map[job_id] = i

    route_nodes = []
    route_map = {}
    for i, (route, forecast) in enumerate(
            associations_df[['route_key', 'forecast_id']].drop_duplicates().itertuples(index=False)):
        route_data = associations_df[
            (associations_df['route_key'] == route) &
            (associations_df['forecast_id'] == forecast)
            ].iloc[0]
        route_nodes.append([route_data['throughput_gbps'], forecast])
        route_map[(route, forecast)] = i + len(job_map)

    # Create edges
    edges = []
    edge_attrs = []
    for _, row in associations_df.iterrows():
        src = job_map[row['job_id']]
        dst = route_map.get((row['route_key'], row['forecast_id']), -1)
        if dst >= 0:
            edges.append((src, dst))
            edge_attrs.append([
                row['carbon_emissions'],
                row['forecast_id'],
                row['throughput_gbps']
            ])

    # Convert to tensors
    x = torch.tensor(job_nodes + route_nodes, dtype=torch.float)
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attrs, dtype=torch.float)

    # Create masks
    job_mask = torch.zeros(x.size(0), dtype=torch.bool)
    job_mask[:len(job_nodes)] = True
    route_mask = ~job_mask

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr,
                job_mask=job_mask, route_mask=route_mask), job_map, route_map


def gnn_optimize(associations_df, epochs=100):
    """End-to-end GNN optimization pipeline"""
    # Prepare graph data
    data, job_map, route_map = prepare_gnn_data(associations_df)
    model = ScheduleGNN()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Inverse mappings
    idx_to_job = {v: k for k, v in job_map.items()}
    idx_to_route = {v: k for k, v in route_map.items()}

    # Training loop
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        scores = model(data)

        # Custom loss - encourages low-carbon, high-utilization schedules
        carbon_loss = (scores.sigmoid() * data.edge_attr[:, 0]).mean()  # Carbon term
        utilization_loss = -scores.sigmoid().mean()  # Utilization term
        loss = carbon_loss + 0.5 * utilization_loss

        # Backprop
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

    # Generate final schedule
    schedule = []
    used_timeslots = defaultdict(set)

    with torch.no_grad():
        scores = model(data)
        edge_order = torch.argsort(scores, descending=True)

        for idx in edge_order:
            src, dst = data.edge_index[:, idx]
            job_id = idx_to_job[src.item()]
            route_key, forecast = idx_to_route[dst.item()]

            if (job_id not in {x['job_id'] for x in schedule}) and (forecast not in used_timeslots[route_key]):
                schedule.append({
                    'job_id': job_id,
                    'route_key': route_key,
                    'forecast_id': forecast,
                    'carbon_emissions': data.edge_attr[idx, 0].item(),
                    'throughput_gbps': data.edge_attr[idx, 2].item()
                })
                used_timeslots[route_key].add(forecast)

    return pd.DataFrame(schedule)


# Usage
associations_df = pd.read_csv("data/associations_df.csv")
schedule = gnn_optimize(associations_df)

print("GNN-Optimized Schedule:")
print(schedule.sort_values(['route_key', 'forecast_id']))
print(f"\nTotal Carbon: {schedule['carbon_emissions'].sum():.2f}")
print(f"Jobs Scheduled: {len(schedule)}/{len(associations_df['job_id'].unique())}")

Epoch 0, Loss: 434.1384
Epoch 10, Loss: 0.0001
Epoch 20, Loss: 0.0000
Epoch 30, Loss: 0.0000
Epoch 40, Loss: 0.0000
Epoch 50, Loss: 0.0000
Epoch 60, Loss: 0.0000
Epoch 70, Loss: 0.0000
Epoch 80, Loss: 0.0000
Epoch 90, Loss: 0.0000
GNN-Optimized Schedule:
     job_id  route_key  forecast_id  carbon_emissions  throughput_gbps
0        86   chi_buff            0          0.753953     3.077484e-09
2        47   chi_buff            1          0.768376     3.077484e-09
4        97   chi_buff            2          0.773151     6.154967e-09
6       107   chi_buff            3          0.768185     1.098870e-05
8        31   chi_buff            4          0.762296     1.434103e-06
..      ...        ...          ...               ...              ...
137     113  tacc_buff           68        289.544586     1.157179e-01
139      92  tacc_buff           69        455.995422     1.158106e-01
144     117  tacc_buff           70       5991.294434     1.159787e-01
145      14  tacc_buff           71

In [83]:
job_id = 1
print(schedule[schedule['job_id'] == job_id])
print(associations_df[(associations_df['route_key'] == 'tacc_buff') & (associations_df['forecast_id'] == 61) & (
            associations_df['job_id'] == 1)][['carbon_emissions', 'throughput']])


     job_id  route_key  forecast_id  carbon_emissions  throughput_gbps
141       1  tacc_buff           61       1428.787476         0.115935
       carbon_emissions    throughput
11011        1428.78742  1.159353e+08


In [111]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GATv2Conv
import torch.nn.functional as F
from torch_geometric.data import Data
import pandas as pd
from collections import defaultdict


class GnnPlanner:
    def __init__(self, associations_df, job_list, model):
        self.associations_df = associations_df
        self.job_list = job_list
        self.job_sizes = {job['id']: job['bytes'] for job in self.job_list}
        self.job_deadlines = {job['id']: job['deadline'] for job in self.job_list}

        self.route_list = associations_df['route_key'].unique()
        self.time_slots = sorted(associations_df['forecast_id'].unique())
        self.max_slot = max(self.time_slots)
        self.epochs = 100
        self.data, self.job_map, self.route_map = self.prepare_gnn_data()
        self.model = model

        # Store throughput (bps) and carbon for output calculations
        self.throughput = dict(zip(
            zip(associations_df['route_key'], associations_df['forecast_id']),
            associations_df['throughput']  # Using throughput in bps
        ))
        self.carbon = dict(zip(
            zip(associations_df['route_key'], associations_df['forecast_id']),
            associations_df['carbon_emissions']
        ))

    def plan(self):
        schedule = self.gnn_optimize()
        print("GNN-Optimized Schedule:")
        print(schedule.sort_values(['job_id', 'forecast_id']))
        print(f"\nTotal Carbon: {schedule['carbon_emissions'].sum():.2f}")
        print(f"Total Allocated Bytes: {schedule['allocated_bytes'].sum():.2f}")

    def prepare_gnn_data(self):
        """Convert dataframe to graph for GNN processing"""
        # Create nodes
        job_nodes = []
        job_map = {}
        for i, job_id in enumerate(self.associations_df['job_id'].unique()):
            job_nodes.append([
                self.job_deadlines[job_id],  # deadline
                self.job_sizes[job_id],  # size in bytes
                0  # current progress
            ])
            job_map[job_id] = i

        route_nodes = []
        route_map = {}
        for i, (route, forecast) in enumerate(
                self.associations_df[['route_key', 'forecast_id']].drop_duplicates().itertuples(index=False)):
            route_data = self.associations_df[
                (self.associations_df['route_key'] == route) &
                (self.associations_df['forecast_id'] == forecast)
                ].iloc[0]
            route_nodes.append([
                route_data['throughput'],  # throughput in bps
                forecast,  # time slot
                route_data['carbon_emissions']  # carbon intensity
            ])
            route_map[(route, forecast)] = i + len(job_map)

        print(f"Route Nodes: {route_nodes}")
        print(f"Route Maps: {route_map}")
        # Create edge
        edges = []
        edge_attrs = []
        for _, row in self.associations_df.iterrows():
            src = job_map[row['job_id']]
            dst = route_map.get((row['route_key'], row['forecast_id']), -1)
            if dst >= 0:
                edges.append((src, dst))
                edge_attrs.append([
                row['carbon_intensity'],  # Just the intensity (gCO2/kWh), not total emissions
                row['forecast_id'],
                row['throughput'],
                self.job_sizes[row['job_id']]
            ])

        # Convert to tensors
        x = torch.tensor(job_nodes + route_nodes, dtype=torch.float)
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_attrs, dtype=torch.float)

        # Create masks
        job_mask = torch.zeros(x.size(0), dtype=torch.bool)
        job_mask[:len(job_nodes)] = True
        route_mask = ~job_mask

        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr,
                    job_mask=job_mask, route_mask=route_mask), job_map, route_map

    def gnn_optimize(self):
        """End-to-end GNN optimization pipeline with fractional allocations"""
        optimizer = optim.Adam(self.model.parameters(), lr=0.001)

        # Inverse mappings
        idx_to_job = {v: k for k, v in self.job_map.items()}
        idx_to_route = {v: k for k, v in self.route_map.items()}

        # Training loop
        for epoch in range(self.epochs):
            self.model.train()
            optimizer.zero_grad()

            # Forward pass
            allocation_scores = self.model(self.data)

            # Custom loss function
            carbon_loss = (allocation_scores.sigmoid() * self.data.edge_attr[:, 0]).mean()

            deadline_loss = self.calculate_deadline_loss(allocation_scores.sigmoid())
            utilization_loss = -allocation_scores.sigmoid().mean()
            print(f"Carbon Loss: {carbon_loss}, Deadline Loss: {deadline_loss}, Utilization Loss: {utilization_loss}")
            loss = carbon_loss + deadline_loss + 0.5 * utilization_loss

            # Backprop
            loss.backward()
            optimizer.step()

            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

        # Generate final schedule with fractional allocations
        schedule = []
        job_progress = {job_id: 0.0 for job_id in self.job_map.keys()}

        with torch.no_grad():
            # Get allocation scores for all edges
            allocation_scores = self.model(self.data).sigmoid()

            # Process each job and allocate fractions to different time slots
            for job_idx, job_id in idx_to_job.items():
                total_size = self.job_sizes[job_id]
                remaining = total_size - job_progress[job_id]

                if remaining <= 0:
                    continue

                # Get all possible allocations for this job
                job_edges = (self.data.edge_index[0] == job_idx).nonzero().squeeze()
                if job_edges.dim() == 0:
                    continue

                # Sort edges by score and process them
                sorted_edges = job_edges[torch.argsort(allocation_scores[job_edges], descending=True)]

                for edge_idx in sorted_edges:
                    if remaining <= 0:
                        break

                    src, dst = self.data.edge_index[:, edge_idx]
                    route_key, forecast = idx_to_route[dst.item()]

                    # Calculate maximum possible allocation for this slot
                    # throughput is in bps, job size in bytes (8 bits per byte)
                    max_possible_bytes = min(remaining, self.data.edge_attr[
                        edge_idx, 2] * 3600 / 8)  # Throughput in bps * 3600s -> bits/hour -> bytes
                    x_val = min(1.0, max_possible_bytes / (self.data.edge_attr[edge_idx, 2] * 3600 / 8))

                    if x_val > 0.01:  # Only consider meaningful allocations
                        schedule.append({
                            'job_id': job_id,
                            'forecast_id': int(forecast),
                            'route_key': route_key,
                            'allocated_fraction': x_val,
                            'allocated_bytes': x_val * self.data.edge_attr[edge_idx, 2] * 3600 / 8,
                            # bps * s -> bits -> bytes
                            'carbon_emissions': x_val * self.data.edge_attr[edge_idx, 0],
                            'completed': (job_progress[job_id] + x_val * max_possible_bytes) >= total_size * 0.99
                        })

                        job_progress[job_id] += x_val * max_possible_bytes
                        remaining = total_size - job_progress[job_id]

        return pd.DataFrame(schedule)

    def calculate_deadline_loss(self, allocations):
        """Penalize allocations that exceed job deadlines"""
        deadline_loss = 0.0
        for job_idx, job_id in self.job_map.items():
            job_edges = (self.data.edge_index[0] == job_idx).nonzero().squeeze()
            if job_edges.dim() == 0:
                continue

            deadline = self.data.x[job_idx, 0]
            late_allocations = allocations[job_edges] * (self.data.edge_attr[job_edges, 1] > deadline).float()
            deadline_loss += late_allocations.sum()

        return deadline_loss / len(self.job_map)


class ScheduleGNN(nn.Module):
    def __init__(self, node_feat_dim=3, edge_feat_dim=4, hidden_dim=128):
        super().__init__()
        # Node encoders
        self.job_encoder = nn.Sequential(
            nn.Linear(node_feat_dim, hidden_dim),
            nn.ReLU()
        )
        self.route_encoder = nn.Sequential(
            nn.Linear(node_feat_dim, hidden_dim),
            nn.ReLU()
        )

        # Graph attention
        self.conv1 = GATv2Conv(hidden_dim, hidden_dim, edge_dim=edge_feat_dim)
        self.conv2 = GATv2Conv(hidden_dim, hidden_dim, edge_dim=edge_feat_dim)

        # Scoring network
        self.scorer = nn.Sequential(
            nn.Linear(2 * hidden_dim + edge_feat_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, data):
        # Encode nodes
        h_jobs = self.job_encoder(data.x[data.job_mask])
        h_routes = self.route_encoder(data.x[data.route_mask])

        # Combine features
        x = torch.zeros(data.num_nodes, h_jobs.size(1), device=data.x.device)
        x[data.job_mask] = h_jobs
        x[data.route_mask] = h_routes

        # Graph processing
        x = F.relu(self.conv1(x, data.edge_index, data.edge_attr))
        x = F.relu(self.conv2(x, data.edge_index, data.edge_attr))

        # Score edges
        src, dst = data.edge_index
        edge_feats = torch.cat([x[src], x[dst], data.edge_attr], dim=1)
        return self.scorer(edge_feats).squeeze()

In [112]:
import json

with open('config/jobs_config/150_jobs.json', 'r') as f:
    job_data = json.load(f)

planner = GnnPlanner(associations_df, job_data, ScheduleGNN())
# planner.plan()

Route Nodes: [[530751779.3365115, 0, 385.34340169836463], [530751779.3365115, 1, 391.20114880727095], [530751779.3365115, 2, 392.1385295392405], [530751779.3365115, 3, 389.46985934915926], [530751779.3365115, 4, 386.7971645398832], [530751779.3365115, 5, 384.01156569359387], [530751779.3365115, 6, 378.9259643536116], [530751779.3365115, 7, 372.8158641385102], [530751779.3365115, 8, 369.0922285947288], [530751779.3365115, 9, 372.6150930433119], [530751779.3365115, 10, 378.20197227270495], [530751779.3365115, 11, 384.491703083236], [530751779.3365115, 12, 379.67419056873257], [530751779.3365115, 13, 361.6185973407164], [530751779.3365115, 14, 350.168463297091], [530751779.3365115, 15, 345.0880716904645], [530751779.3365115, 16, 339.79225918773386], [530751779.3365115, 17, 333.75494047101483], [530751779.3365115, 18, 327.4079264907392], [530751779.3365115, 19, 322.55194175870804], [530751779.3365115, 20, 319.2727466585075], [530751779.3365115, 21, 325.3653930401583], [530751779.3365115, 2